In [1]:
!pip install -q kaggle

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ./kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [4]:
!kaggle datasets download -d nursyahrina/chat-sentiment-dataset
!unzip chat-sentiment-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/nursyahrina/chat-sentiment-dataset
License(s): copyright-authors
  0% 0.00/7.42k [00:00<?, ?B/s]
100% 7.42k/7.42k [00:00<00:00, 12.0MB/s]
Archive:  chat-sentiment-dataset.zip
  inflating: chat_dataset.csv        


In [5]:
import pandas as pd
import numpy as np

In [6]:
df1 = pd.read_csv('/content/chat_dataset.csv')
df1.head()

,message,sentiment
0,I really enjoyed the movie,positive
1,The food was terrible,negative
2,I'm not sure how I feel about this,neutral
3,The service was excellent,positive
4,I had a bad experience,negative


In [10]:
data = np.zeros([100,4])
data = pd.DataFrame(data, columns=['AA UCID', 'TRANSCRIPT_CHUNK/Trunk', 'Question', 'Intent Identified'])

In [11]:
ucid = [100, 200, 300, 400] * 25
ucid.sort()
ucid = np.array(ucid)

chat = df1.message.iloc[:80]
chat = np.append(chat, np.repeat(np.nan, 20))
np.random.shuffle(chat)

intent = ['Yes - Chit Chat', 'Yes - Review Medical', 'Yes - background Info', 'No - Background Info'] * 25
intent = np.array(intent)
np.random.shuffle(intent)

question = df1.message.iloc[100:130]
question = np.append(question, np.repeat(np.nan, 70))
np.random.shuffle(question)

data['AA UCID'] = ucid
data['TRANSCRIPT_CHUNK/Trunk'] = chat
data['Question'] = question
data['Intent Identified'] = intent
data

,AA UCID,TRANSCRIPT_CHUNK/Trunk,Question,Intent Identified
0,100,The scenery here is average,I have no strong opinion on this,Yes - background Info
1,100,I had a bad experience,NaN,No - Background Info
2,100,The museum was boring,NaN,Yes - background Info
3,100,I don't really care about this,I regret buying this,Yes - Review Medical
4,100,The food was average,The service at this restaurant was mediocre,Yes - Chit Chat
...,...,...,...,...
95,400,I regret buying this product,This movie was disappointing,No - Background Info
96,400,This product is overpriced,The beach was dirty,Yes - background Info
97,400,This book is a masterpiece,I don't really have an opinion on this,Yes - background Info
98,400,The scenery here is beautiful,NaN,Yes - Review Medical


In [48]:
def test(data, rolling=0):
  all_data = []
  for ucid_number in data['AA UCID'].unique():
    data_temp = data[data['AA UCID'] == ucid_number]
    first_index = data_temp.index[0]
    if rolling>0:
      for i in range(data_temp.shape[0]):
        temp_chat = ' '.join(data_temp.iloc[i:i+rolling]['TRANSCRIPT_CHUNK/Trunk'].dropna().tolist())
        final_question = (data_temp['Question'][i:i+rolling].dropna().tolist())
        final_intent = (data_temp[~data_temp['Intent Identified'].str.contains('Chit', regex=True, na=False)].loc[first_index+i:first_index+i+rolling]['Intent Identified'].unique().tolist())
        all_data.append([ucid_number, temp_chat, final_question, final_intent])


    else:
      final_chat = ' '.join(data_temp['TRANSCRIPT_CHUNK/Trunk'].dropna().tolist())
      final_question = data_temp['Question'].dropna().tolist()
      final_intent = data_temp[~data_temp['Intent Identified'].str.contains('Chit', regex=True, na=False)]['Intent Identified'].unique().tolist()
      all_data.append([ucid_number, final_chat, final_question, final_intent])

  all_data_df = pd.DataFrame(all_data, columns=['AA UCID', 'TRANSCRIPT_CHUNK/Trunk', 'Question', 'Intent Identified'])
  return all_data_df
all_data = test(data, 4)
all_data

,AA UCID,TRANSCRIPT_CHUNK/Trunk,Question,Intent Identified
0,100,The scenery here is average I had a bad experi...,"[I have no strong opinion on this, I regret bu...","[Yes - background Info, No - Background Info, ..."
1,100,I had a bad experience The museum was boring I...,"[I regret buying this, The service at this res...","[No - Background Info, Yes - background Info, ..."
2,100,The museum was boring I don't really care abou...,"[I regret buying this, The service at this res...","[Yes - background Info, Yes - Review Medical, ..."
3,100,I don't really care about this The food was av...,"[I regret buying this, The service at this res...","[Yes - Review Medical, No - Background Info]"
4,100,The food was average I wouldn't recommend this...,"[The service at this restaurant was mediocre, ...","[No - Background Info, Yes - Review Medical]"
...,...,...,...,...
95,400,I regret buying this product This product is o...,"[This movie was disappointing, The beach was d...","[No - Background Info, Yes - background Info, ..."
96,400,This product is overpriced This book is a mast...,"[The beach was dirty, I don't really have an o...","[Yes - background Info, Yes - Review Medical]"
97,400,This book is a masterpiece The scenery here is...,[I don't really have an opinion on this],"[Yes - background Info, Yes - Review Medical]"
98,400,The scenery here is beautiful,[],[Yes - Review Medical]
